In [130]:
puzzle = open('puzzle.txt', 'r').read()
example = open('example.txt', 'r').read()

In [131]:
input = puzzle

box_positions = input.split('\n')

# Part 1

In [132]:
import numpy as np

def get_dist(xyz1,xyz2):
    x1,y1,z1 = xyz1[0],xyz1[1],xyz1[2]
    x2,y2,z2 = xyz2[0],xyz2[1],xyz2[2]
    return np.sqrt((x1-x2)**2 + (y1-y2)**2 + (z1-z2)**2)

In [133]:
# Initialise the distances between all pairs of boxes

box_dists = {}
for box1 in box_positions:
    for box2 in box_positions:
        # Skip self and repeat occurrences
        if box1 == box2 or (box1,box2) in box_dists or (box2,box1) in box_dists:
            continue
        else:
            x1,y1,z1 = map(int,(box1.split(',')))
            x2,y2,z2 = map(int,(box2.split(',')))
        box_dists[(box1,box2)] = get_dist((x1,y1,z1),(x2,y2,z2))

# Sort box distances by largest -> smallest
box_dists = dict(sorted(box_dists.items(), key=lambda item: item[1]))

# Total connections to make
N = 1000

# Keep track of box connections
box_connections = {}

for connection_count,boxes in enumerate(box_dists):
    if connection_count >= N:
        break
    
    box1,box2 = boxes[0],boxes[1]

    if box1 in box_connections:
        box_connections[box1] = box_connections[box1]+[box2]
    else:
        box_connections[box1] = [box2]

    if box2 in box_connections:
        box_connections[box2] = box_connections[box2]+[box1]
    else:
        box_connections[box2] = [box1]

# Keep track of sizes of box connections
connection_sizes = []

# Initialise boxes left to check the connections for
remaining_boxes = set(box_connections.keys())

# Iterate until we have checked every box and its connection
while(remaining_boxes):
    # Focus on the connection that next_box is part of
    next_box = remaining_boxes.pop()
    remaining_boxes_in_connection = set(box_connections[next_box])

    # Track all boxes in this connection
    this_seen = set([next_box])

    # Iterate through the boxes we know are in this connection
    while(remaining_boxes_in_connection):
        next_box_in_connection = remaining_boxes_in_connection.pop()

        # Add current box to the ones seen in this connection
        if next_box_in_connection not in this_seen:
            this_seen.add(next_box_in_connection)

        # Add the boxes connected to this box to the remaining boxes to check this connection
        for next_box_in_connection_connection in box_connections[next_box_in_connection]:
            if next_box_in_connection_connection not in this_seen:
                remaining_boxes_in_connection.add(next_box_in_connection_connection)
    
    connection_sizes.append(len(this_seen))

    for box in this_seen:
        if box in remaining_boxes:
            remaining_boxes.remove(box)



sorted_sizes = sorted(connection_sizes, reverse=True)
(sorted_sizes[0]*sorted_sizes[1]*sorted_sizes[2])

42315

# Part 2

In [ ]:
# Repeat the connection map initialisation from part 1:

import numpy as np

def get_dist(xyz1,xyz2):
    x1,y1,z1 = xyz1[0],xyz1[1],xyz1[2]
    x2,y2,z2 = xyz2[0],xyz2[1],xyz2[2]
    return np.sqrt((x1-x2)**2 + (y1-y2)**2 + (z1-z2)**2)

# Initialise the distances between all pairs of boxes

box_dists = {}
for box1 in box_positions:
    for box2 in box_positions:
        # Skip self and repeat occurrences
        if box1 == box2 or (box1,box2) in box_dists or (box2,box1) in box_dists:
            continue
        else:
            x1,y1,z1 = map(int,(box1.split(',')))
            x2,y2,z2 = map(int,(box2.split(',')))
        box_dists[(box1,box2)] = get_dist((x1,y1,z1),(x2,y2,z2))

# Sort box distances by largest -> smallest
box_dists = dict(sorted(box_dists.items(), key=lambda item: item[1]))





# The plan:
# - Make a function F that we can feed a connection map into that returns the largest connection
# - Do a binary search on the number of connections to make
# where we make the stopping criteria F(x-1) < 1000 and F(x) == 1000

def largest_connection(box_connections):
    # Keep track of sizes of box connections
    connection_sizes = []

    # Initialise boxes left to check the connections for
    remaining_boxes = set(box_connections.keys())

    # Iterate until we have checked every box and its connection
    while(remaining_boxes):
        # Focus on the connection that next_box is part of
        next_box = remaining_boxes.pop()
        remaining_boxes_in_connection = set(box_connections[next_box])

        # Track all boxes in this connection
        this_seen = set([next_box])

        # Iterate through the boxes we know are in this connection
        while(remaining_boxes_in_connection):
            next_box_in_connection = remaining_boxes_in_connection.pop()

            # Add current box to the ones seen in this connection
            if next_box_in_connection not in this_seen:
                this_seen.add(next_box_in_connection)

            # Add the boxes connected to this box to the remaining boxes to check this connection
            for next_box_in_connection_connection in box_connections[next_box_in_connection]:
                if next_box_in_connection_connection not in this_seen:
                    remaining_boxes_in_connection.add(next_box_in_connection_connection)
        
        connection_sizes.append(len(this_seen))

        for box in this_seen:
            if box in remaining_boxes:
                remaining_boxes.remove(box)
        
    return sorted(connection_sizes,reverse=True)[0]


def construct_box_connections(connections_to_make, return_final_boxes=False):
    # Keep track of box connections
    box_connections = {}

    for connection_count,boxes in enumerate(box_dists):
        if connection_count >= connections_to_make:
            break
        
        box1,box2 = boxes[0],boxes[1]

        if box1 in box_connections:
            box_connections[box1] = box_connections[box1]+[box2]
        else:
            box_connections[box1] = [box2]

        if box2 in box_connections:
            box_connections[box2] = box_connections[box2]+[box1]
        else:
            box_connections[box2] = [box1]

    if return_final_boxes:
        return box1,box2
    
    return box_connections


# Starting guess
connections_to_make = 1000

lower = 100
upper = 10000

connections = largest_connection(construct_box_connections(connections_to_make))
connections_one_less = largest_connection(construct_box_connections(connections_to_make-1))

while(not (connections_one_less < 1000 and connections >= 1000)):

    if connections_one_less >= 1000:
        upper = connections_to_make
    else:
        lower = connections_to_make

    print(connections_to_make, connections_one_less, connections)
    
    connections_to_make = ((lower+upper)//2)

    connections = largest_connection(construct_box_connections(connections_to_make))
    connections_one_less = largest_connection(construct_box_connections(connections_to_make-1))


connections_to_make

1000 39 39
5500 999 999
7750 1000 1000
6625 1000 1000
6062 999 999
6343 1000 1000
6202 999 999
6272 1000 1000
6237 1000 1000
6219 1000 1000
6210 1000 1000
6206 1000 1000


6204

In [136]:
# 6203 -> 6204
construct_box_connections(6204, return_final_boxes=True)


('97682,37616,383', '82710,37374,3897')

In [137]:
97682*82710

8079278220